<a href="https://colab.research.google.com/github/asarria48/Nuclear-physics/blob/main/Previa2ASM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import curve_fit
import os

# Para ver varios espectros

In [ ]:

# Nombres de los archivos .csv que quiero comparar
archivos = [
    "Cs137_5minNaI.csv",
    "Co60_10minNaI.csv",
    "Na22_10minNaI.csv"
]

colores = ['black', 'steelblue', 'green']
etiquetas = ['Cs', 'Co', 'Na']

plt.figure(figsize=(12, 5))


for archivo, color, etiqueta in zip(archivos, colores, etiquetas):
    data = pd.read_csv(archivo)
    x = data.iloc[:, 0].values   # primera columna: canal
    y = data.iloc[:, 1].values   # segunda columna: cuentas
    plt.plot(x, y, lw=1.3, color=color, label=etiqueta)

plt.xlabel("Canal", fontsize=14)
plt.ylabel("[cuentas/canal]", fontsize=14)
plt.grid(alpha=0.3)
plt.legend(fontsize=14)
plt.tight_layout()
plt.show()

# Para ver un espectro

In [1]:
datos = "2021-11-15_cal_133Ba22Na137Cs60Co_600s_1.csv"
df = pd.read_csv(datos)

x = df.iloc[:, 0]
y = df.iloc[:, 1]

plt.figure(figsize=(10,6))
plt.plot(x, y, color='purple', ds="steps-mid", lw=1, label="Espectro: Na, Ba, Co, Cs.")
plt.tick_params(axis='both', which='major', labelsize=12)
plt.tick_params(axis='both', which='minor', labelsize=12)
plt.yscale('log')
plt.xlabel("Canal", fontsize=14)
plt.ylabel("[cuentas/canal]", fontsize=14)
plt.grid(which="both", linestyle="--", linewidth=0.5, alpha=0.6)
plt.xlim(0,3000)
plt.ylim(1e0, 1e5)

plt.legend(fontsize=14)
plt.tight_layout()
plt.show()


NameError: name 'pd' is not defined

# Para ajustar picos

In [ ]:
def gauss_bg(x, mu, sigma, A, a0, a1):                                              # gaussiana con fondo lineal
    return A * np.exp(-0.5 * ((x - mu) / sigma)**2) + a0 + a1 * x

def fwhm(sigma): return 2.3458 * sigma
def dfwhm(dsigma): return 2.3458 * dsigma
def intensidad(A, sigma): return A * sigma * np.sqrt(2*np.pi)
def dintensidad(A, sigma, dA, dsigma):
    return np.sqrt((sigma*np.sqrt(2*np.pi)*dA)**2 + (A*np.sqrt(2*np.pi)*dsigma)**2)

def format_unc(value, error):
    if error == 0 or np.isnan(error):
        return f"{value:.4g}"
    exp_err = int(np.floor(np.log10(abs(error))))
    digits = max(0, -exp_err + 1)
    value_rounded = round(value, digits)
    err_rounded = int(round(error * 10**(-exp_err + 1)))
    if digits > 0:
        return f"{value_rounded:.{digits}f}({err_rounded})"
    else:
        return f"{int(round(value_rounded))}({err_rounded})"


# ajustes en general
def ajustar_pico(x, y, x_min, x_max, color="purple"):

    mask = (x >= x_min) & (x <= x_max)
    xr, yr = x[mask], y[mask]

    mu_guess = xr.iloc[np.argmax(yr)]
    A_guess = yr.max()
    p0 = [mu_guess, 4.0, A_guess, np.median(yr), 0.0]

    popt, pcov = curve_fit(gauss_bg, xr, yr, p0=p0, maxfev=10000)
    perr = np.sqrt(np.diag(pcov))

    mu, sigma, A, a0, a1 = popt
    dmu, dsigma, dA, da0, da1 = perr

    print(f"\n======")
    labels = ["μ", "σ", "FWHM", "Int"]
    vals   = [mu, sigma, fwhm(sigma), intensidad(A, sigma)]
    errs   = [dmu, dsigma, dfwhm(dsigma), dintensidad(A, sigma, dA, dsigma)]

    for l, v, e in zip(labels, vals, errs):
        print(f"{l:8s} = {format_unc(v, e)}")

    xf = np.linspace(xr.min(), xr.max(), 800)
    fit = gauss_bg(xf, *popt)
    bg  = a0 + a1 * xf

    # gráficas
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.step(xr, yr, where='mid', color='black', lw=0.8)
    ax.plot(xf, fit, color=color, lw=2, label='Ajuste')
    ax.plot(xf, bg, color='steelblue', lw=1.2, label='Fondo')
    ax.fill_between(xf, bg, fit, where=(fit > bg), color=color, alpha=0.3)


    ax.set_xlabel("Canal", fontsize=14)
    ax.set_ylabel("[cuentas/canal]", fontsize=14)
    ax.legend(fontsize=12)
    ax.grid(alpha=0.3)


    plt.tight_layout()
    plt.show()


# ajusto varios picos de una vez

datos = "2021-11-15_cal_133Ba22Na137Cs60Co_600s_1.csv"
df = pd.read_csv(datos)

x = df.iloc[:, 0]
y = df.iloc[:, 1]

picos = [
    (156, 185, "purple"),                                                      # del bario 133, 79.61 keV
    (565, 590, "purple"),                                                      # del bario 133, 276.39 keV
    (615, 650, "purple"),                                                      # del bario 133, 302.85 keV
    (730, 760, "purple"),                                                      # del bario 133, 356.01 keV
    (785, 815, "purple"),                                                      # del bario 133, 383.84 keV
    (1365, 1392, "purple"),                                                    # del cesio 137 (no el pico de rayos X) 661 keV
    (2430, 2455, "purple"),                                                    # del cobalto 60 (primero) 1173 keV
    (2630, 2670, "purple"),                                                    # del cobalto 60 (segundo) 1332 keV
    (2755, 2790, "purple"),                                                    # del sodio 22, 1275 keV
]

for xmin, xmax, color in picos:
    ajustar_pico(x, y, xmin, xmax, color)

# Para calibrar energía

In [ ]:
E = np.array([79.6139, 276.3997, 302.8510, 356.0134, 383.8480, 661.657, 1173.228, 1274.537, 1332.490])
mu = np.array([170.065, 576.718, 631.826, 742.409, 800.281, 1378.142, 2441.608, 2652.062, 2772.424])
dmu = np.array([0.045, 0.063, 0.055, 0.059, 0.059, 0.049, 0.053, 0.066, 0.073])
dE = np.ones_like(E)


def f(E, b0, b1):
    return b0 + b1 * E                                                                                # canal = b0 + b1 * energía

popt, pcov = curve_fit(f, E, mu, sigma=dE, absolute_sigma=True)
b0, b1 = popt
db0, db1 = np.sqrt(np.diag(pcov))

print("=== Ajuste de canal vs energía ===")
print(f"b0 = {b0:.3f} ± {db0:.3f}")
print(f"b1 = {b1:.5f} ± {db1:.5f}")
print(pcov[0,1])

# invierto
# E = a0 + a1 * canal
a1 = 1 / b1
a0 = -b0 / b1
da1 = db1 / b1**2
da0 = np.sqrt((db0/b1)**2 + (b0*db1/b1**2)**2)


def format_uncertainty(value, uncertainty):
    n_decimals = int(-np.floor(np.log10(uncertainty))) + 1
    value_rounded = round(value, n_decimals)
    uncertainty_rounded = round(uncertainty, n_decimals)
    digits = str(uncertainty_rounded).replace('.', '')[:3]
    return f"{value_rounded:.{n_decimals}f}({digits})"

a0_str = format_uncertainty(a0, da0)
a1_str = format_uncertainty(a1, da1)

print("\n=== Resultados de la calibración en energía ===")
print(f"a0 = {a0_str} keV")
print(f"a1 = {a1_str} keV/canal")
print(f"\nEcuación de calibración:")
print(f"E = {a0_str} + {a1_str} * canal")
print(f"Acomodar la incertidumbre de a0 a (3) y despreciar la de a1.")


plt.figure(figsize=(10,5))
plt.errorbar(mu, E, xerr=dmu, fmt='o', color='black', capsize=3, label='Datos')
plt.plot(mu, a0 + a1*mu, color='green', lw=1, label='Ajuste lineal')
plt.xlabel('Canal', fontsize=14)
plt.ylabel('[keV]', fontsize=14)
plt.legend(fontsize=14)
plt.grid(True, ls='--', alpha=0.6)
plt.tight_layout()
plt.show()


E_ajustado = a0 + a1 * mu
dif = E - E_ajustado
error_rel = np.abs(dif / E) * 100

# print("\n=== Comparación Energía Teórica vs Ajustada ===")
# print(f"{'Canal':>8} {'E_teórico (keV)':>18} {'E_ajustado (keV)':>20} {'ΔE (keV)':>12} {'Error %':>10}")
# for i in range(len(E)):
#     print(f"{mu[i]:8.2f} {E[i]:18.3f} {E_ajustado[i]:20.3f} {dif[i]:12.3f} {error_rel[i]:10.3f}")

# Para econtrar la energía relacionada al canal

In [ ]:
# cambio de acuerdo a lo que salga de la calibración de energía
canal = 75
a0 = 12
a1 = 13
E = a0 + a1 * canal
print(E)

# Eje x en keV

In [ ]:
datos = "2021-11-17_mue_IAEA-RGK-1_24h.csv"
df = pd.read_csv(datos)

canal = df.iloc[:, 0]
cuentas = df.iloc[:, 1]

# cambiar de acuerdo a la calibración que extraiga

a0 = -1.41     # keV
a1 = 0.48114   # keV/canal

energia = a0 + a1 * canal   # conversión canal → keV


plt.figure(figsize=(10,5))
plt.plot(energia, cuentas, color='midnightblue', ds="steps-mid", lw=1, label='Referencia K')
plt.tick_params(axis='both', which='major', labelsize=12)
plt.tick_params(axis='both', which='minor', labelsize=12)
plt.yscale('log')
plt.xlabel("[keV]", fontsize=14)
plt.ylabel("I_\\gamma[cuentas/canal]", fontsize=14)
plt.grid(which="both", linestyle="--", linewidth=0.5, alpha=0.6)
plt.legend(fontsize=14)
plt.xlim(0,1600)
plt.ylim(1, 1e6)
plt.tight_layout()
plt.show()

# Dibujar bordes compton y de retrodispersión

In [ ]:
archivo = "Co60_10minNaI.csv"                                                         # cambio entre .csv
data = pd.read_csv(archivo)                                                           # columna 0: canal, columna 1: cuentas

x = data.iloc[:,0].values
y = data.iloc[:,1].values

retrodispersión = 565
borde = 2250

# Teóricos (keV)
E_C1 = 963.1
E_R1 = 209.8
E_C2 = 1118.1
E_R2 = 214.3

# Experimentales (keV)
E_C1x = 911.3
E_R1x = 218.8
E_R2x = 220.8

# para ver el espectro completo
plt.figure(figsize=(14,5))
plt.plot(x, y, lw=0.8, color='black', label='Co')
plt.axvline(x=retrodispersión, color='purple', lw=2, linestyle='--', label='Retrodispersión')
plt.axvline(x=borde, color='green', lw=2,linestyle='--', label='Borde Compton')
plt.text(retrodispersión + 50, max(y)*0.9, f'Canal {retrodispersión}', color='purple', fontsize=14)
plt.text(borde - 500, max(y)*0.9, f'Canal {borde}', color='green', fontsize=14)
plt.xlabel("Canal", fontsize=14)
plt.ylabel("[cuentas/canal]", fontsize=14)
plt.legend(fontsize=14)
plt.grid(True)
plt.show()


# para ver el espectro completo
plt.figure(figsize=(14,5))
plt.plot(x, y, lw=0.8, color='black', label='Co')
plt.axvline(x=retrodispersión, color='purple', lw=2, linestyle='--', label='Retrodispersión')
plt.axvline(x=borde, color='green', lw=2,linestyle='--', label='Borde Compton')
plt.text(retrodispersión + 50, max(y)*0.9, f'Canal {retrodispersión}', color='purple', fontsize=14)
plt.text(borde - 500, max(y)*0.9, f'Canal {borde}', color='green', fontsize=14)
plt.xlabel("Canal", fontsize=14)
plt.ylabel("[cuentas/canal]", fontsize=14)
plt.legend(fontsize=14)
plt.grid(True)
plt.show()


#Restar el fondo de los espectros

In [ ]:
archivos = [
    "NaI_137Cs_600s.dat",
    "NaI_22Na_600s.dat",
    "NaI_60Co_600s.dat",
    "NaI_57Co_300s.dat",
    "NaI_Fondo_600s.dat",
    "NaI_Fondo_300s.dat"
]

colores = ['black', 'steelblue', 'green', 'darkorange', 'gray', 'gray']
etiquetas = ['
Cs (600 s)', '
Na (600 s)', '
Co (600 s)', '
Co (300 s)', 'Fondo (600 s)', 'Fondo (300 s)']

def leer_dat(archivo):
    with open(archivo, 'r', encoding='latin-1') as f:
        lineas = f.readlines()
    lineas_numericas = [l for l in lineas if l.strip() and l.strip()[0].isdigit()]
    from io import StringIO
    data = pd.read_csv(StringIO(''.join(lineas_numericas)), sep=r'\s+', header=None)
    data.columns = ['canal', 'cuentas']
    return data

# los fondos
fondo_600 = leer_dat("NaI_Fondo_600s.dat")
fondo_300 = leer_dat("NaI_Fondo_300s.dat")

plt.figure(figsize=(12, 5))


for archivo, color, etiqueta in zip(archivos, colores, etiquetas):
    if "Fondo" in archivo:                                                      # ya no grafico los fondos. Además, distingo espectros de fondos
        continue

    data = leer_dat(archivo)
    x = data['canal'].values
    y = data['cuentas'].astype(float).values


    if "600s" in archivo:                                                       # si el espectro es de 600 segundos
        y_fondo = np.interp(x, fondo_600['canal'], fondo_600['cuentas'])
        y -= y_fondo                                                            # le resto el respectivo fondo de 600 segundos
    elif "300s" in archivo:                                                     # si el espectro es de 300 segundos
        y_fondo = np.interp(x, fondo_300['canal'], fondo_300['cuentas'])
        y -= y_fondo                                                            # le resto el respectivo fondo de 300 segundos

    plt.plot(x, y, lw=1.3, color=color, label=etiqueta)

plt.xlabel("Canal", fontsize=14)
plt.ylabel("
 [cuentas/canal]", fontsize=14)
plt.xlim(0, 500)
plt.grid(alpha=0.3)
plt.legend(fontsize=14)
plt.tight_layout()
plt.show()


data_na22 = leer_dat("NaI_22Na_600s.dat")                                       # como la intensidad es tan baja para el sodio, conviene más mostrarla por separado
x = data_na22['canal'].values
y = data_na22['cuentas'].astype(float).values
y_fondo = np.interp(x, fondo_600['canal'], fondo_600['cuentas'])
y -= y_fondo

plt.figure(figsize=(10, 4))
plt.plot(x, y, color='steelblue', lw=1.5)
plt.xlabel("Canal", fontsize=13)
plt.ylabel("
 [cuentas/canal]", fontsize=13)
plt.xlim(0, 500)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Ajuste gaussiano
(otra vez pero con energía)

In [ ]:

def gauss_bg(x, mu, sigma, A, a0, a1):                                          # gaussiana con fondo lineal
    return A * np.exp(-0.5 * ((x - mu) / sigma)**2) + a0 + a1 * x

def fwhm(sigma): return 2.3458 * sigma
def dfwhm(dsigma): return 2.3458 * dsigma
def intensidad(A, sigma): return A * sigma * np.sqrt(2*np.pi)
def dintensidad(A, sigma, dA, dsigma):
    return np.sqrt((sigma*np.sqrt(2*np.pi)*dA)**2 + (A*np.sqrt(2*np.pi)*dsigma)**2)

def format_unc(value, error):
    if error == 0 or np.isnan(error):
        return f"{value:.4g}"
    exp_err = int(np.floor(np.log10(abs(error))))
    digits = max(0, -exp_err + 1)
    value_rounded = round(value, digits)
    err_rounded = int(round(error * 10**(-exp_err + 1)))
    if digits > 0:
        return f"{value_rounded:.{digits}f}({err_rounded})"
    else:
        return f"{int(round(value_rounded))}({err_rounded})"


# ajustes en general
def ajustar_pico(x, y, x_min, x_max, color="purple", energia_keV=None):

    mask = (x >= x_min) & (x <= x_max)
    xr, yr = x[mask], y[mask]

    mu_guess = xr.iloc[np.argmax(yr)]
    A_guess = yr.max()
    p0 = [mu_guess, 4.0, A_guess, np.median(yr), 0.0]

    popt, pcov = curve_fit(gauss_bg, xr, yr, p0=p0, maxfev=10000)
    perr = np.sqrt(np.diag(pcov))

    mu, sigma, A, a0, a1 = popt
    dmu, dsigma, dA, da0, da1 = perr

    print(f"\n======")
    labels = ["μ", "σ", "FWHM", "Int"]
    vals   = [mu, sigma, fwhm(sigma), intensidad(A, sigma)]
    errs   = [dmu, dsigma, dfwhm(dsigma), dintensidad(A, sigma, dA, dsigma)]

    for l, v, e in zip(labels, vals, errs):
        print(f"{l:8s} = {format_unc(v, e)}")

    xf = np.linspace(xr.min(), xr.max(), 800)
    fit = gauss_bg(xf, *popt)
    bg  = a0 + a1 * xf

    # gráficas
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.step(xr, yr, where='mid', color='black', lw=0.8)
    ax.plot(xf, fit, color=color, lw=2, label='Ajuste')
    ax.plot(xf, bg, color='steelblue', lw=1.2, label='Fondo')
    ax.fill_between(xf, bg, fit, where=(fit > bg), color=color, alpha=0.3)

    if energia_keV is not None:
     ax.text(
            0.05, 0.90,
            f"{energia_keV:.2f} keV",
            transform=ax.transAxes,
            fontsize=13,
            color="black",
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
        )

    ax.set_xlabel("Canal", fontsize=14)
    ax.set_ylabel("
 [cuentas/canal]", fontsize=14)
    ax.legend(fontsize=12)
    ax.grid(alpha=0.3)


    plt.tight_layout()
    plt.show()


# ajusto varios picos de una vez

datos = "2021-11-15_cal_133Ba22Na137Cs60Co_600s_1.csv"
df = pd.read_csv(datos)

x = df.iloc[:, 0]
y = df.iloc[:, 1]

picos = [
    (156, 185, "purple", 79.61),                                                        # del bario 133, 79.61 keV
    (565, 590, "purple", 276.39),                                                       # del bario 133, 276.39 keV
    (615, 650, "purple", 302.85),                                                       # del bario 133, 302.85 keV
    (730, 760, "purple", 356.01),                                                       # del bario 133, 356.01 keV
    (785, 815, "purple", 383.84),                                                       # del bario 133, 383.84 keV
    (1365, 1392, "purple", 661.65),                                                     # del cesio 137 (no el pico de rayos X) 661 keV
    (2430, 2455, "purple", 1173.22),                                                    # del cobalto 60 (primero) 1173 keV
    (2630, 2670, "purple", 1332.49),                                                    # del cobalto 60 (segundo) 1332 keV
    (2755, 2790, "purple", 1274.53),                                                    # del sodio 22, 1275 keV
]

for xmin, xmax, color, energia in picos:
    ajustar_pico(x, y, xmin, xmax, color, energia)
